In [1]:
import requests
import pandas
from bs4 import BeautifulSoup

In [2]:
# initial input
account = '106010'
password = 'G122117609'
session = requests.session()
chracter_choose = 'cloud' # could choose "cloud" or "yourself"
ticket_type = 'handling'
search_column = 'disp_seq'
query_string = ''
max_count = None

In [3]:
chracters = {
    'cloud': '20',
    'yourself': account,
}

In [4]:
def login_with_given_session(account, password, session):
    post_url = 'http://202.3.168.17:8080/login_check.jsp'
    post_data = {
        'sess_id': account,
        'sess_password': password,
        'dfForm': 'login.jsp',
    }
    http_response = session.post(url=post_url, data=post_data)
    return http_response

In [5]:
login_with_given_session(account=account, password=password, session=session)

<Response [200]>

In [6]:
def search_on_dashboard_terminal(session, character, search_column, query_string, max_count):
    post_url = 'http://202.3.168.17:8080/Disp/DashBoard_Terminal.jsp?action=search'
    post_data = {
        'recordCount': max_count,
        'searchColumn': search_column,
        'searchCondition': query_string,
        'doSearch': 'Search',
        'character': character,
    }
    http_response = session.post(url=post_url, data=post_data)
    return http_response

In [7]:
search_http_response = search_on_dashboard_terminal(
    session=session,
    character=chracters[chracter_choose],
    search_column=search_column,
    query_string=query_string,
    max_count=max_count
)

In [8]:
def get_dataframe_list_by_reading_html(html):
    return pandas.read_html(html)

In [9]:
ticket_tables = get_dataframe_list_by_reading_html(search_http_response.text)

In [10]:
# take the part we want as a dictionary
ticket_tables = {
    'preordered': ticket_tables[2],
    'assigned': ticket_tables[3],
    'handling': ticket_tables[4],
    'finished': ticket_tables[5],
    'special': ticket_tables[6],
}

In [11]:
def let_the_first_row_be_column_title(dataframe):
    new_dataframe = dataframe[1:]
    new_dataframe.columns = dataframe.iloc[0]
    return new_dataframe

In [12]:
def clean_the_column_of_ticket_number(dataframe):
    for index, row in dataframe.iterrows():
        row['單號'] = row['單號'].replace(' (Delay)', '')
    return dataframe

In [13]:
for key, value in ticket_tables.items():
    value = let_the_first_row_be_column_title(value)
    value = clean_the_column_of_ticket_number(value)
    ticket_tables[key] = value

In [14]:
def get_ticket_file_url_with_given_ticket_number(ticket_number, session):
    post_url = 'http://202.3.168.17:8080/Disp/DashBoard_Terminal_Detail.jsp'
    post_data = {
        'seq_no': ticket_number,
    }
    response = session.post(url=post_url, data=post_data)
    soup = BeautifulSoup(response.text, 'lxml')
    tag_set = soup.select('body > table > tr:nth-child(2) > td > fieldset > table > tr:nth-child(1) > td > ol > li > input[type=button]:nth-child(1)')
    if len(tag_set) == 1:
        tag = tag_set[0]
    else:
        tag = None
    if tag:
        url = tag['onclick'].split(',')[0].split('"')[1]
    else:
        url = ''
    return url

In [15]:
ticket_number_list = list(ticket_tables[ticket_type]['單號'])

In [16]:
urls = [ get_ticket_file_url_with_given_ticket_number(ticket, session) for ticket in ticket_number_list ]

In [18]:
ticket_tables[ticket_type].assign(建置單檔案=urls)

,單號,客戶名稱,協銷客戶名稱,類別,專案,流程,建置單檔案
1,202003240627,格威傳媒股份有限公司,NaN,新增/測試單,李瑞賢,處理中,http://202.3.168.75:8080/build_report.jsp?tw_f...
2,202004290985,中信房屋仲介股份有限公司,NaN,新增/測試單,李瑞賢,處理中,http://202.3.168.75:8080/build_report.jsp?tw_f...
3,202004270937,中國旺旺控股有限公司,NaN,新增/測試單,李瑞賢,處理中,http://202.3.168.75:8080/build_report.jsp?tw_f...
4,202004220892,海博資訊股份有限公司,NaN,新增/測試單,李瑞賢,處理中,http://202.3.168.75:8080/build_report.jsp?tw_f...
5,202004210873,傑思網擎有限公司,NaN,異動/測試單,李瑞賢,處理中,http://202.3.168.75:8080/build_report.jsp?tw_f...
6,202004210872,鈊象電子股份有限公司,NaN,新增/測試單,李瑞賢,處理中,http://202.3.168.75:8080/build_report.jsp?tw_f...
7,202004210871,OLYMPIC STAR LIMITED,NaN,新增/測試單,李瑞賢,處理中,http://202.3.168.75:8080/build_report.jsp?tw_f...
8,202004170860,發現資訊有限公司,NaN,新增/測試單,李瑞賢,處理中,http://202.3.168.75:8080/build_report.jsp?tw_f...
9,202004150826,TY solution,NaN,新增/測試單,李瑞賢,處理中,http://202.3.168.75:8080/build_report.jsp?tw_f...
10,202004100791,雲一有限公司,[建置單: ] [案管:IGS-3],新增/測試單,李忠憲,處理中,http://202.3.168.75:8080/build_report.jsp?tw_f...


In [19]:
def get_ticket_detail_with_given_ticket_number(session, chracter, ticket_number):
    post_url = 'http://202.3.168.17:8080/Disp/retriveDetail.jsp'
    post_data = {
        'method': 'get_Disp_DetailCons',
        'Disp_Cons_Seq': chracter,
        'Disp_Grp_Seq': chracter,
        'Disp_Seq': ticket_number,
    }
    http_response = session.post(url=post_url, data=post_data)
    return http_response

In [20]:
df_list = []
for ticket_number in list(ticket_tables[ticket_type]['單號']):
    ticket_detail_response = get_ticket_detail_with_given_ticket_number(session=session, chracter=chracters[chracter_choose], ticket_number=ticket_number)
    df_ticket_detail = get_dataframe_list_by_reading_html(ticket_detail_response.text)[0]
    df_ticket_detail = let_the_first_row_be_column_title(df_ticket_detail)
    df_ticket_detail = df_ticket_detail.assign(單號=ticket_number)
    df_list += [df_ticket_detail]

In [21]:
df_left = pandas.concat(df_list)
df_right = ticket_tables[ticket_type]
db_joined = pandas.merge(df_left, df_right, on='單號', how='outer', suffixes=('', '-細項'))

In [23]:
db_joined['建置單檔案']

KeyError: '建置單檔案'